In [10]:
import numpy as np

from keras.models import Sequential
from keras.layers import Embedding, Dense
from keras.layers.recurrent import LSTM

from data.models import SentenceBatch, Task
from keras_ml.models import Model, TrainSession
from word_embeddings.utils import load_w2v_model
from keras import backend as K

from tqdm import tqdm_notebook
import itertools

from django.db.models import Max

In [28]:
s_ids = TrainSession.objects.filter(
    batch__task=2
).values(
    'model_id'
).annotate(
    f1_score=Max('f1_score')
).order_by('-f1_score').values_list('id', flat=True)[:3]
s_ids

<QuerySet [2722, 2099, 2100]>

In [29]:
TrainSession.objects.filter(id__in=s_ids)

<QuerySet [<TrainSession: #2722, 29, Aspect detection, entity: DRINKS, attribute: PRICES>, <TrainSession: #2099, 1, Aspect detection, entity: DRINKS, attribute: PRICES>, <TrainSession: #2100, 1, Aspect detection, entity: DRINKS, attribute: PRICES>]>

In [7]:
sessions[0].f1_score

0.850439882697947

In [38]:
runs = 10

In [39]:
for task in tqdm_notebook(Task.objects.all()):
    best_session_ids = TrainSession.objects.filter(
        batch__task=task
    ).values(
        'model_id'
    ).annotate(
        f1_score=Max('f1_score')
    ).order_by('-f1_score').values_list('id')[:3]
    
    sessions = TrainSession.objects.filter(id__in=best_session_ids)
    
    for session in sessions:
        for _ in range(runs):
            TrainSession.objects.create(
                model=session.model,
                batch=session.batch,
                train_opts=session.train_opts
            )

HBox(children=(IntProgress(value=0, max=22), HTML(value='')))

In [40]:
TrainSession.objects.filter(f1_score__isnull=True).delete()

(660, {'keras_ml.TrainSession': 660})

In [ ]:
for batch in tqdm_notebook(SentenceBatch.objects.filter(task__type=Task.Type.ASPECT_DETECTION)):
    for _ in range(runs):
        TrainSession.objects.create(
            model=keras_model,
            batch=batch,
            train_opts=train_opts
        )

In [ ]:
Model.objects.count()

In [ ]:
activations = ['softmax', 'elu', 'selu', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
activation_pairs = []
for activation, recurrent_activation in tqdm_notebook(itertools.product(activations, repeat=2)):
    K.clear_session()
    # Create model
    embedding_layer = Embedding(
    input_dim=len(w2v_model.wv.vocab),
    output_dim=w2v_model.vector_size,
    input_length=len(batch.x_train[0]),
    mask_zero=True
    )

    lstm_layer = LSTM(
        units=200,
        activation=activation,
        recurrent_activation=recurrent_activation
    )
    dense_layer = Dense(1, activation='sigmoid')

    model = Sequential()
    model.add(embedding_layer)
    model.add(lstm_layer)
    model.add(dense_layer)

    config = model.get_config()
    compile_opts = dict(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    train_opts = dict(
        epochs=10,
        validation_split=0.1,
        batch_size=128
    )
    
    keras_model, _ = Model.objects.get_or_create(
        config=model.get_config(),
        compile_opts=compile_opts
    )


In [ ]:
for model in tqdm_notebook(Model.objects.all()):
    for batch in SentenceBatch.objects.filter(task__type=Task.Type.ASPECT_DETECTION):
        TrainSession.objects.create(
            model=model,
            batch=batch,
            train_opts=train_opts
        )

In [ ]:
# POLARITY DETECTION

In [10]:
for batch in tqdm_notebook(SentenceBatch.objects.filter(task__type=Task.Type.POLARITY_DETECTION)):
    for _ in range(1):
        TrainSession.objects.create(
            model=keras_model,
            batch=batch,
            train_opts=train_opts
        )

HBox(children=(IntProgress(value=0, max=3), HTML(value='')))